In [5]:
#importing Libraries
import pandas as pd
import selenium
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
import numpy as np
import time
import os
import urllib
import re

In [6]:
#importing driver
driver = webdriver.Chrome(r"C:\Users\Vishvendra Singh\Desktop\chromedriver.exe")


# 1. Write a python program which searches all the product under a particular product vertical from www.amazon.in. The product verticals to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

# 2. In the above question, now scrape the following details of each product listed in first 3 pages

In [3]:
def search_amazon(item):

    driver.get('https://www.amazon.in')
    search_box = driver.find_element_by_id('twotabsearchtextbox').send_keys(item)
    search_button = driver.find_element_by_id("nav-search-submit-text").click()
    
    #number of pages
    num = np.arange(1,4)
    
    #Making List
    ref=[]
    brands = []
    names = [] 
    ratings = []
    num_ratings =[]
    prices = []
    exchanges = []
    expected = []
    availability =[]
    others = []
    product_url = []
    
    for page in num:
        urls=driver.find_elements_by_xpath("//div[@class='a-section a-spacing-medium']/div[2]/h2/a")
        nxt_button=driver.find_element_by_xpath("//li[@class='a-last']/a")
        next_arr= nxt_button.get_attribute('href')
        for i in urls:
            ref.append(i.get_attribute('href'))
        for url in ref:
            driver.get(url)
            time.sleep(2)
            #Scrapping Brand Name
            try:
                if driver.find_element_by_xpath("//th[@class='a-color-secondary a-size-base prodDetSectionEntry']").text == 'Brand':
                    brand = driver.find_element_by_xpath("//td[@class='a-size-base prodDetAttrValue']")
                    brands.append(brand.text) 
                else:
                    brand = driver.find_element_by_xpath("//td[@class='a-span9']")
                    brands.append(brand.text)
            except NoSuchElementException as e:
                brands.append("--")

            #Scrapping Product Info
            try:
                product = driver.find_element_by_id('productTitle')
                names.append(product.text)
            except NoSuchElementException as e:
                names.append("--")

            #Scrapping Ratings
            try:
                rating = driver.find_element_by_xpath("//span[@class='a-size-base a-nowrap']/span")
                ratings.append(rating.text)
            except NoSuchElementException as e:
                ratings.append("--")

            # Scrapping Number of Ratings
            try:
                num_rating = driver.find_element_by_id('acrCustomerReviewText')
                num_ratings.append(num_rating.text)
            except NoSuchElementException as e:
                num_ratings.append("--")

            #Scrapping Replacement
            try:
                exchange = driver.find_element_by_xpath("//div[@id='RETURNS_POLICY']")
                exchanges.append(exchange.text)
            except NoSuchElementException as e:
                exchanges.append("--")

            #Scrapping Price
            try:
                price = driver.find_element_by_xpath("//td[@class='a-span12']/span[1]")
                prices.append(price.text)
            except NoSuchElementException as e:
                prices.append("--")


            #Scrapping Delivery
            try:
                expect = driver.find_element_by_xpath("//div[@id='ddmDeliveryMessage']/b")
                expected.append(expect.text)
            except NoSuchElementException as e:
                expected.append("--")


             #Scrapping Availability
            try:
                avail = driver.find_element_by_xpath("//div[@id='availability']")
                availability.append(avail.text)
            except NoSuchElementException as e:
                availability.append("--")

            #Scrapping Other Details
            try:
                other = driver.find_element_by_xpath("//div[@id='featurebullets_feature_div']")
                others.append(other.text.replace('\n', ''))
            except NoSuchElementException as e:
                others.append("--")

            #Screpping URL
            product_url.append(url)


        driver.get(next_arr)
        time.sleep(2)
        print("Number of pages :-", page)
    #making dataframe
    ds = pd.DataFrame({'Brand' : brands ,'names' : names ,
                         'ratings' :ratings ,'num_ratings' :num_ratings ,
                         'prices' :prices ,'exchanges' :exchanges ,
                         'expected' :expected ,'availability' :availability ,
                         'others' :others ,'product_url' :product_url})
    return ds


In [4]:
product = search_amazon(input('Enter Product Name :- '))

Enter Product Name :- guitar
Number of pages :- 1
Number of pages :- 2
Number of pages :- 3


In [5]:
product

,Brand,names,ratings,num_ratings,prices,exchanges,expected,availability,others,product_url
0,Kadence,Kadence Frontier Jumbo Semi Acoustic Guitar Wi...,3.8 out of 5,281 ratings,"₹ 5,999.00",7 Days Replacement,"Monday, March 1",In stock.,QUALITY STRINGS ---The surface is coated with ...,https://www.amazon.in/gp/slredirect/picassoRed...
1,Medellin,Medellin MED-BLU-C Linden Wood Acoustic Guitar,3.8 out of 5,253 ratings,"₹ 2,399.00",7 Days Replacement,"Monday, March 1",In stock.,Material: WoodColour: BlueAcoustic Guitar› See...,https://www.amazon.in/gp/slredirect/picassoRed...
2,Giuson,Giuson Electra Rosewood Fretboard Acoustic Gui...,5 out of 5,1 rating,"₹ 3,790.00",7 Days Replacement,Feb 28 - March 5,"In stock on February 26, 2021.\nOrder it now.",Inbuilt pick up to plug and play with amplifie...,https://www.amazon.in/gp/slredirect/picassoRed...
3,1Stopper,1Stopper 4 String Guitar Children's Musical In...,--,--,₹ 731.00,7 Days Replacement,Feb 27 - 28,In stock.,PROVIDES SCREEN FREE FUN: A junior scale 4 str...,https://www.amazon.in/gp/slredirect/picassoRed...
4,Juârez,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...",3.9 out of 5,"7,291 ratings","₹ 2,359.00",7 Days Replacement,"Friday, Feb 26",In stock.,"About this itemBlack Glossy Finish, Number of ...",https://www.amazon.in/Juarez-Acoustic-Cutaway-...
...,...,...,...,...,...,...,...,...,...,...
351,SHOPECOM,SHOPECOM String Acoustic Guitar | Adjustable T...,--,--,₹ 899.00,7 Days Replacement,"Saturday, Feb 27",In stock.,1. This Acoustic Guitar makes a great gift for...,https://www.amazon.in/SHOPECOM-Acoustic-Adjust...
352,HRB MUSICALS,HRB MUSICALS GUITAR FLOOR STAND +GUITAR CAPO +...,3.4 out of 5,2 ratings,₹ 649.00,7 Days Replacement,"Friday, Feb 26",In stock.,"For guitar floor stand its solid body ,string ...",https://www.amazon.in/HRB-MUSICALS-GUITAR-STRI...
353,Blueberry,"Blueberry B-40, 40"" Acoustic Guitar Kit with B...",2.6 out of 5,3 ratings,"₹ 3,595.00",7 Days Replacement,"Thursday, Feb 25",In stock.,"Material: Top: Spruce wood, Side & Back: - lin...",https://www.amazon.in/Blueberry-Acoustic-Guita...
354,HOVNER,Hovner 215 Red F-Cut Rosewood Fretboard Acoust...,--,--,"₹ 3,980.00",7 Days Replacement,"Sunday, Feb 28",In stock.,"41 inch F- Cut Acoustic Guitar, Fretboard : Ro...",https://www.amazon.in/215-Rosewood-Fretboard-A...


# 3. Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.


In [4]:
#name of folder
folder = 'Result Images'

#making folder to save the video
def main():
    if not os.path.exists(folder):
        os.mkdir(folder)
    download_images()
    

def download_images():
    
    #taking input
    item = input("What do you want to search for? ")
    n_images = int(input('How many images do you want? '))
    
    #searching URL
    driver.get('https://images.google.com/?gws_rd=ssl')
    
    #clicking Search button
    search_box = driver.find_element_by_xpath("//input[@class='gLFyf gsfi']").send_keys(item)
    search_button = driver.find_element_by_xpath("//span[@class='z1asCe MZy1Rb']").click()
    
    #Scrolling 10 Times
    value = 0
    for i in range(10):
        driver.execute_script("scrollBy("+ str(value) +",+1000);")
        value += 1000
        time.sleep(1)
    time.sleep(5)
    
    #extracting required element
    image_class=driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']")
    
    
    count = 0
    print('Downloading',n_images,'images of',item,'...' )
    for j,i in enumerate(image_class):
        if j<n_images:
            image_arr= i.get_attribute('src')
            try:
                if image_arr != None:
                    image_arr= str(image_arr)
                    
                    #saving image to folder
                    urllib.request.urlretrieve(image_arr, os.path.join(folder, str(j)+'.jpg'))
                else:
                    raise TypeError
            except Exception as e: 
                print(f'fail with error {e}')
    
    print('Downloading Done')


In [5]:
if __name__ == "__main__":
    main() 

What do you want to search for? fruits
How many images do you want? 100


In [6]:
if __name__ == "__main__":
    main() 

What do you want to search for? cars
How many images do you want? 100


In [10]:
if __name__ == "__main__":
    main() 

What do you want to search for? machine learning
How many images do you want? 100


# 4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. 

In [13]:
def main():
    
    #taking input
    item = input("Enter name of Mobile :- ")
    
    #Searching URL
    driver.get('https://www.flipkart.com')

    #clicking Pop-up Cross Button
    try:
        cross_btn = driver.find_element_by_xpath("//button[@class='_2KpZ6l _2doB4z']")
        cross_btn.click() 
    except NoSuchElementException as e:
        pass
    
    #finding  different element
    search = driver.find_element_by_xpath("//div[@class='_3OO5Xc']/input")
    search.send_keys( item + 'mobile')

    #clicking Search button
    search_btn = driver.find_element_by_xpath("//button[@class='L0Z3Pu']")
    search_btn.click()
    
    #Making List
    ref=[]
    brands = []
    names = [] 
    colors = []
    rams =[]
    storages = []
    pcameras = []
    scameras = []
    dsizes =[]
    dres = []
    process = []
    process_cores = []
    batteries =[]
    prices = []
    product_url = []


    time.sleep(2)
    value = 0
    for i in range(10):
        driver.execute_script("scrollBy("+ str(value) +",+1000);")
        value += 1000

    urls=driver.find_elements_by_xpath("//div[@class='_1AtVbE col-12-12']/div/div/div/a")
    
    for i in urls:
        ref.append(str(i.get_attribute('href')))
    for url in ref:
        driver.get(url)
        time.sleep(2)
        driver.find_element_by_xpath("//button[@class='_2KpZ6l _1FH0tX']").click()
        #Scrapping Brand Name
        try:
            brand = driver.find_element_by_xpath("//h1[@class='yhB1nd']/span")
            brands.append(brand.text.split()[0])
        except NoSuchElementException as e:
            brands.append("-")

        #Scrapping Model Number
        try:
            product = driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[1]/table/tbody/tr[3]/td[2]")
            names.append(product.text)
        except NoSuchElementException as e:
            names.append("-")

        #Scrapping Color
        try:
            color = driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[1]/table/tbody/tr[4]/td[2]")
            colors.append(color.text)
        except NoSuchElementException as e:
            colors.append("-")

        # Scrapping RAM
        try:
            ram = driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[4]/table/tbody/tr[2]/td[2]")
            rams.append(ram.text)
        except NoSuchElementException as e:
            rams.append("-")

        # Scrapping ROM
        try:
            storage = driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[4]/table/tbody/tr[1]/td[2]")
            storages.append(storage.text)
        except NoSuchElementException as e:
            storages.append("-")

        # Scrapping Primary Camera
        try:
            pcamera = driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[5]/table/tbody/tr[2]/td[2]")
            pcameras.append(pcamera.text)
        except NoSuchElementException as e:
            pcameras.append("-")

        # Scrapping Secondary Camera
        try:
            scamera = driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[5]/table/tbody/tr[5]/td[2]")
            scameras.append(scamera.text)
        except NoSuchElementException as e:
            scameras.append("-")

        #Scrapping Display Size
        try:
            dsize = driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[2]/table/tbody/tr[1]/td[2]")
            dsizes.append(dsize.text)
        except NoSuchElementException as e:
            dsizes.append("-")

        #Scrapping Display Resolution
        try:
            dre = driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[2]/table/tbody/tr[2]/td[2]")
            dres.append(dre.text)
        except NoSuchElementException as e:
            dres.append("-")


        #Scrapping Processor Name
        try:
            proces = driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[3]/table/tbody/tr[2]/td[2]")
            process.append(proces.text)
        except NoSuchElementException as e:
            process.append("-")


        #Scrapping Processor Number of Cores
        try:
            process_core = driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[3]/table/tbody/tr[3]/td[2]")
            process_cores.append(process_core.text)
        except NoSuchElementException as e:
            process_cores.append("-")

        #Scrapping Battery
        try:
            if driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[10]/div").text == 'Battery & Power Features':
                battery = driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[10]/table/tbody/tr[1]/td[2]")
                batteries.append(battery.text)
            else:
                batteries.append("-")
        except NoSuchElementException as e:
            batteries.append("-")

        #Scrapping Price
        try:
            price = driver.find_element_by_xpath("//div[@class='dyC4hf']/div[1]/div/div[1]")
            prices.append(price.text)
        except NoSuchElementException as e:
            prices.append("-")

        #Screpping URL
        product_url.append(url)
        
    #making dataframe
    ds = pd.DataFrame({'Brand' : brands ,'Names' : names ,'Color':colors ,'RAM': rams, 'ROM' : storages, 
                       'Primary Camera' : pcameras, 'Secondary Camera' : scameras,'Display Size':dsizes, 
                       'Display Resulation':dres, 'Processor':process, 'Core':process_cores, 'Battery':batteries,
                       'Price':prices,'product_url' :product_url})
    
    
    
    #saving dataframe to csv
    ds.to_csv('mobile_list.csv')
    
    return ds
if __name__ == "__main__":
    main() 

df= pd.read_csv('mobile_list.csv')
df

Enter name of Mobile :- redmi


,Unnamed: 0,Brand,Names,Color,RAM,ROM,Primary Camera,Secondary Camera,Display Size,Display Resulation,Processor,Core,Battery,Price,product_url
0,0,Redmi,Redmi 9 Prime,Space Blue,4 GB,64 GB,13MP Rear Camera,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 Pixels,MediaTek Helio G80,Octa Core,5020 mAh,"₹9,499",https://www.flipkart.com/redmi-9-prime-space-b...
1,1,Redmi,Redmi 9 Prime,Sunrise Flare,4 GB,64 GB,13MP Rear Camera,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 Pixels,MediaTek Helio G80,Octa Core,5020 mAh,"₹9,499",https://www.flipkart.com/redmi-9-prime-sunrise...
2,2,Redmi,Redmi 9 Prime,Matte Black,4 GB,64 GB,13MP Rear Camera,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 Pixels,MediaTek Helio G80,Octa Core,5020 mAh,"₹9,499",https://www.flipkart.com/redmi-9-prime-matte-b...
3,3,Redmi,Redmi 9 Power,Mighty Black,4 GB,64 GB,48MP Rear Camera,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080$$pixel,Qualcomm Snapdragon 662,Octa Core,6000 mAh,"₹10,965",https://www.flipkart.com/redmi-9-power-mighty-...
4,4,Redmi,Redmi 9 Prime,Mint Green,4 GB,128 GB,13MP Rear Camera,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 Pixels,MediaTek Helio G80,Octa Core,5020 mAh,"₹11,983",https://www.flipkart.com/redmi-9-prime-mint-gr...
5,5,Redmi,Redmi Note 8,Neptune Blue,4 GB,64 GB,48MP + 8MP + 2MP + 2MP,13MP Front Camera,16.0 cm (6.3 inch),2340 x 1080 Pixels,Qualcomm Snapdragon 665,Octa Core,4000 mAh,"₹12,999",https://www.flipkart.com/redmi-note-8-neptune-...
6,6,Redmi,Redmi 9 Power,Electric Green,4 GB,64 GB,48MP Rear Camera,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080$$pixel,Qualcomm Snapdragon 662,Octa Core,6000 mAh,"₹11,209",https://www.flipkart.com/redmi-9-power-electri...
7,7,Redmi,Redmi Note 9,Aqua Green,6 GB,128 GB,48MP Rear Camera,13MP AI Selfie Camera,16.59 cm (6.53 inch),2340 x 1080$$pixel,MediaTek Helio G85,Octa Core,-,"₹14,645",https://www.flipkart.com/redmi-note-9-aqua-gre...
8,8,Redmi,Note 9 Pro Max,Interstellar Black,6 GB,64 GB,64MP + 8MP + 5MP + 2MP,32MP Front Camera,16.94 cm (6.67 inch),2400 x 1080 Pixels,Qualcomm Snapdragon 720G,Octa Core,-,"₹14,999",https://www.flipkart.com/redmi-note-9-pro-max-...
9,9,Redmi,Note 9 Pro Max,Champagne Gold,6 GB,64 GB,64MP + 8MP + 5MP + 2MP,32MP Front Camera,16.94 cm (6.67 inch),2400 x 1080 Pixels,Qualcomm Snapdragon 720G,Octa Core,-,"₹14,999",https://www.flipkart.com/redmi-note-9-pro-max-...


# 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [5]:
def main():
    
    
    place = input('Enter Name Of Place :- ')
    
    #Searching URL
    driver.get('https://www.google.com/maps')
    
    #Searching
    search = driver.find_element_by_xpath("//div[@id='gs_lc50']/input")
    search.send_keys(place)
    
    #clicking Search button
    search_btn = driver.find_element_by_xpath("//button[@id='searchbox-searchbutton']")
    search_btn.click()
    
    print('Getting Location...')
    time.sleep(10)
    
    
    #Getting Current URL
    curr_url = driver.current_url
    
    #Scrapping results from URl through Regular Expression Library
    Latitude=float(re.findall("(?<=@)\d{1,3}\.\d{5,}",str(curr_url))[0])
    Longitude= float(re.findall("(?<=,)\d{1,3}\.\d{5,}",str(curr_url))[0])
    
    print('Latitide of ',place,' =',Latitude)
    print('Longitude of ',place,' =',Longitude)
    
if __name__ == "__main__":
    main() 
    

Enter Name Of Place :- Noida
Getting Location...
Latitide of  Noida  = 26.6282755
Longitude of  Noida  = 73.873897


# 6. Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 – September 20) from trak.in.

In [8]:
#Getting Link
driver.get('https://trak.in/')

#navigating to Funding Deals
l1 = driver.find_element_by_xpath("//li[@id='menu-item-51510']/a").get_attribute('href')
driver.get(l1)

time.sleep(5)
#Selecting 100 entries per page
driver.find_element_by_xpath("//div[@id='tablepress-48_length']/label/select/option[4]").click()
driver.find_element_by_xpath("//div[@id='tablepress-49_length']/label/select/option[4]").click()
driver.find_element_by_xpath("//div[@id='tablepress-50_length']/label/select/option[4]").click()

#downloading page source
html = driver.page_source

#reading tables
df = pd.read_html(html)

#selecting required dataframe
frame = [df[13],df[11],df[9]]

result = pd.concat(frame,ignore_index=True,join="inner")
result

,Sr. No.,Date (dd/mm/yyyy),Startup Name,Industry / Vertical,Sub-Vertical,City / Location,Investors' Name,Investment Type,Amount (In USD)
0,1,15/07/2020,Flipkart,E-commerce,E-commerce,Bangalore,Walmart Inc,M&A,1200000000
1,2,16/07/2020,Vedantu,EduTech,Online Tutoring,Bangalore,Coatue Management,Series D,100000000
2,3,16/07/2020,Crio,EduTech,Learning Platform for Developers,Bangalore,021 Capital,pre-Series A,934160
3,4,14/07/2020,goDutch,FinTech,Group Payments,Mumbai,"Matrix India,Y Combinator, Global Founders Cap...",Seed,1700000
4,5,13/07/2020,Mystifly,Airfare Marketplace,"Ticketing, Airline Retailing, and Post-Ticketi...",Singapore and Bangalore,Recruit Co. Ltd.,pre-Series B,3300000
5,6,09/07/2020,JetSynthesys,Gaming and Entertainment,Gaming and Entertainment,Pune,Adar Poonawalla and Kris Gopalakrishnan.,Venture-Series Unknown,400000
6,7,10/07/2020,gigIndia,Marketplace,"Crowd Sourcing, Freelance",Pune,Incubate Fund India and Beyond Next Ventures,pre-Series A,974200
7,8,15/07/2020,PumPumPum,Automotive Rental,Used Car-leasing platform,Gurgaon,Early Adapters Syndicate,Seed,292800
8,9,14/07/2020,FLYX,OTT Player,Streaming Social Network,New York and Delhi,"Raj Mishra, founder of AIT Global Inc",pre-Seed,200000
9,10,13/07/2020,Open Appliances Pvt. Ltd.,Information Technology,Internet-of-Things Security Solutions,Bangalore,Unicorn India Ventures,Venture-Series Unknown,500000


# Write a program to scrap all the available details of top 10 gaming laptops from digit.in.

In [4]:
#Getting Link
driver.get('https://www.digit.in/')

#Navigating to the link of best gaming laptop
l1 = driver.find_element_by_xpath("//div[@class='listing_container']/ul/li[9]/a").get_attribute('href')
driver.get(l1)

#downloading page source
html = driver.page_source

#reading tables
df = pd.read_html(html)

#Scrapping Specification Tables
num = np.arange(1,11)
l1= []
for i in num:
    df1 = df[1].drop('SPECIFICATION.1', axis = 1)
    df2 = df1.drop('SPECIFICATION', axis = 1)
    ds = df2.transpose()
    
    l1.append(ds)
    
#Joining Specification Tables
result = pd.concat(l1,ignore_index=True,join="inner")
result['Product Name']=df[0]['Product Name']
result['Seller']=df[0]['Seller']
result['Price']=df[0]['Price']

result.drop(7, axis = 1, inplace = True)

#Renaming final Table
result = result.rename(columns={0 : "OS", 1: "Display",2: "Processor",3: "Memory",4: "Weight",5: "Dimension",6: "Graphics Processor"})
result

,OS,Display,Processor,Memory,Weight,Dimension,Graphics Processor,Product Name,Seller,Price
0,Windows 10 Pro,"17.3"" (UHD 3840x2160)",Intel 9th Gen Core i9-9900K | 5000 MHz,1 TB HDD/64 GBGB DDR4,4.2 kg,397 x 330 x 33~42 mm,NVIDIA GeForce RTX 2080,MSI GT76 TITAN DT 9SG,NaN,₹379990
1,Windows 10 Pro,"17.3"" (UHD 3840x2160)",Intel 9th Gen Core i9-9900K | 5000 MHz,1 TB HDD/64 GBGB DDR4,4.2 kg,397 x 330 x 33~42 mm,NVIDIA GeForce RTX 2080,Alienware 17 Area-51m,amazon,₹489909
2,Windows 10 Pro,"17.3"" (UHD 3840x2160)",Intel 9th Gen Core i9-9900K | 5000 MHz,1 TB HDD/64 GBGB DDR4,4.2 kg,397 x 330 x 33~42 mm,NVIDIA GeForce RTX 2080,HP Omen 15 2020,amazon,₹114990
3,Windows 10 Pro,"17.3"" (UHD 3840x2160)",Intel 9th Gen Core i9-9900K | 5000 MHz,1 TB HDD/64 GBGB DDR4,4.2 kg,397 x 330 x 33~42 mm,NVIDIA GeForce RTX 2080,Asus Zephyrus G14,amazon,₹164990
4,Windows 10 Pro,"17.3"" (UHD 3840x2160)",Intel 9th Gen Core i9-9900K | 5000 MHz,1 TB HDD/64 GBGB DDR4,4.2 kg,397 x 330 x 33~42 mm,NVIDIA GeForce RTX 2080,Lenovo Legion Y540,flipkart,₹79990
5,Windows 10 Pro,"17.3"" (UHD 3840x2160)",Intel 9th Gen Core i9-9900K | 5000 MHz,1 TB HDD/64 GBGB DDR4,4.2 kg,397 x 330 x 33~42 mm,NVIDIA GeForce RTX 2080,Asus ROG Zephyrus G GA502,flipkart,₹79990
6,Windows 10 Pro,"17.3"" (UHD 3840x2160)",Intel 9th Gen Core i9-9900K | 5000 MHz,1 TB HDD/64 GBGB DDR4,4.2 kg,397 x 330 x 33~42 mm,NVIDIA GeForce RTX 2080,Asus ROG Zephyrus S GX531,amazon,₹146000
7,Windows 10 Pro,"17.3"" (UHD 3840x2160)",Intel 9th Gen Core i9-9900K | 5000 MHz,1 TB HDD/64 GBGB DDR4,4.2 kg,397 x 330 x 33~42 mm,NVIDIA GeForce RTX 2080,MSI GT83VR 7RE Titan SLI,NaN,₹349990
8,Windows 10 Pro,"17.3"" (UHD 3840x2160)",Intel 9th Gen Core i9-9900K | 5000 MHz,1 TB HDD/64 GBGB DDR4,4.2 kg,397 x 330 x 33~42 mm,NVIDIA GeForce RTX 2080,ASUS ROG ZEPHYRUS DUO 15,amazon,₹259990
9,Windows 10 Pro,"17.3"" (UHD 3840x2160)",Intel 9th Gen Core i9-9900K | 5000 MHz,1 TB HDD/64 GBGB DDR4,4.2 kg,397 x 330 x 33~42 mm,NVIDIA GeForce RTX 2080,Dell G3,amazon,₹61990
